In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os, random, zipfile
import numpy as np
import gdown

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

url = "https://drive.google.com/file/d/1tW4fP2mUqlQtCftq81evUL_Q-uXxsGaI/view?usp=sharing"
file_id = "1tW4fP2mUqlQtCftq81evUL_Q-uXxsGaI"
zip_path = "dataset.zip"
gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_path, quiet=False)

out_dir = "extracted_files"
os.makedirs(out_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(out_dir)

train_path = 'extracted_files/dataset/train'
valid_path = 'extracted_files/dataset/valid'
test_path = 'extracted_files/dataset/test'

train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
valid_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
    valid_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

for layer in vgg_base.layers:
    layer.trainable = False

model_tl = models.Sequential()
model_tl.add(vgg_base)
model_tl.add(layers.Flatten())
model_tl.add(layers.Dense(256, activation='relu'))
model_tl.add(layers.Dropout(0.5))
model_tl.add(layers.Dense(train_batches.num_classes, activation='softmax'))

model_tl.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_tl = model_tl.fit(
    train_batches,
    validation_data=valid_batches,
    epochs=5,
    verbose=1
)